In [1]:
import camb
pars = camb.CAMBparams()
from scipy.special import jn, jn_zeros
from camb import model, initialpower
from scipy.interpolate import interp1d
from hankel_transform import *
import matplotlib.pyplot as plt
import matplotlib
#from nbodykit.cosmology import Planck15
from astropy.cosmology import Planck15 #watch units...
import pickle
from power_spectra import *

## Setup for Sukhdeep's code

In [2]:
def get_camb(zlist,kmin=.8e-3,kmax=30,):
    #Set up a new set of parameters for CAMB
    k_smooth=1
    nk=5000
    non_linear=1

    #assuming these are right
    pars = camb.CAMBparams()
    #This function sets up CosmoMC-like settings, with one massive neutrino and helium set using BBN consistency
    pars.set_cosmology(H0=67.7, ombh2=0.022250, omch2=0.119800, mnu=0.06, omk=0, tau=0.06)
    pars.InitPower.set_params(ns=0.965, r=0,As =2.14e-09)
    pars.set_for_lmax(2500, lens_potential_accuracy=0)
    zb=zlist
    pars.set_matter_power(redshifts=zb,kmax=kmax);
    if non_linear==1:
        pars.NonLinear = model.NonLinear_both
    else:
        pars.NonLinear = model.NonLinear_none
    results = camb.get_results(pars)
    kh, z, pk =results.get_matter_power_spectrum(minkh=kmin, maxkh=kmax, npoints =nk)

    return (kh,z,pk)

In [3]:
#Set up Hankel Transform
#If you only need wgg, set j_nu=[0].
rmin=.6
rmax=110
kmax=30
kmin=.8e-3
%time HT=hankel_transform(rmin=rmin,rmax=rmax,kmax=kmax,j_nu=[0],n_zeros=60000,kmin=kmin)
taper_kw=dict({'large_k_lower':10,'large_k_upper':kmax,'low_k_lower':kmin,'low_k_upper':kmin*1.2})

nr: 1047
CPU times: user 18 s, sys: 482 ms, total: 18.5 s
Wall time: 19 s


In [4]:
r_bins=np.logspace(np.log10(5),np.log10(110),24) 
# ^ remember we want to stop at 5 before hitting data
#^Though this doesn't account for cross-covariance between small and large scales? 
#presumably jackknife "knows" about this?

## 0. Preliminary run

In [135]:
#main
#input data
names = ['lowz','cmass','lrg','elg','qso']
propnames = ['zeff','Ng','Area','depth','pk']
data = np.array([[.27,.63,.8,.85,.95],[45671,74186,24404,89967,7759],[860,860,700,620,700]])
z_edges = [[0.,.5],[.35,.8],[.6,1.],[.6,1.1],[.8,1.1]] #straight from table, maybe not the most realistic
depths = [Planck15.comoving_distance(z[1]).value*Planck15.h #convert to Mpc/h
                   - Planck15.comoving_distance(z[0]).value*Planck15.h 
                   for z in z_edges]
#compute CAMB once
kh,_,pks = get_camb(data[0])
props = [dict(zip(propnames,list(np.concatenate([data[:,i],np.atleast_1d(depths[i])],axis=0))+[pks[i]] )) for i in range(5)]
sdict = dict(zip(names,props))

PS = Power_Spectra(cosmo_h=0.677) #assuming Planck15's agree
Dchi2_gg = 200 #??? #LOS integration length in the estimator.

Note: redshifts have been re-sorted (earliest first)


In [136]:
sdict

{'lowz': {'zeff': 0.27,
  'Ng': 45671.0,
  'Area': 860.0,
  'depth': 1317.9232225273754,
  'pk': array([2.41539290e+03, 2.42018959e+03, 2.42499541e+03, ...,
         5.62995788e-01, 5.60238431e-01, 5.57494089e-01])},
 'cmass': {'zeff': 0.63,
  'Ng': 74186.0,
  'Area': 860.0,
  'depth': 981.8146241626717,
  'pk': array([1.67309651e+03, 1.67641989e+03, 1.67974959e+03, ...,
         3.31475451e-01, 3.29844697e-01, 3.28221687e-01])},
 'lrg': {'zeff': 0.8,
  'Ng': 24404.0,
  'Area': 700.0,
  'depth': 761.5328241925426,
  'pk': array([1.42224415e+03, 1.42506962e+03, 1.42790047e+03, ...,
         2.69852545e-01, 2.68525442e-01, 2.67204637e-01])},
 'elg': {'zeff': 0.85,
  'Ng': 89967.0,
  'Area': 620.0,
  'depth': 925.2666276812847,
  'pk': array([1.35797728e+03, 1.36067518e+03, 1.36337821e+03, ...,
         2.55018194e-01, 2.53764375e-01, 2.52516504e-01])},
 'qso': {'zeff': 0.95,
  'Ng': 7759.0,
  'Area': 700.0,
  'depth': 521.6214500489964,
  'pk': array([1.24066443e+03, 1.24312946e+03, 1.24

In [154]:
#run the code
def run_snr(area=None,name=None,cross=None):
    for i_s,sname in enumerate(sdict):
        
        sample = sdict[sname]
        #compute effective volume
        if(area is None): 
            area = sample['Area']
            
        area_comoving=area*(np.pi/180)**2*(Planck15.comoving_distance(sample['zeff']).value*Planck15.h)**2
        vol=area_comoving*(sample['depth']/2) #not sure what this factor should be but 2 is close to 500 for lowz
        if(name is None):
            n_g=sample['Ng']/vol
        else:
            n_g=sample['Ng_'+name]/vol
        g_shot_noise=1./n_g
        sample['sn'] = g_shot_noise
        bzeff = 1.75/PS.DZ_int([sample['zeff']]) #simple bias as recommended ~2
        p_g=bzeff**2 * sample['pk']

        #compute wgg
        r_gg,wgg=HT.projected_correlation(k_pk=kh,pk=p_g,j_nu=0)
        wgg_re =  np.interp((r_bins[1:]+r_bins[:-1])/2,r_gg,wgg)#"binned" wgg

        #compute gg covariance
        if(cross is not None): 
            #this is gross but it works
            cross_sample = sdict[cross]
            
            #use smaller of the two areas and use width corresponding to smaller effective redshift
            #not sure if this actually matters
            if(area is None): area = min(sample['Area'],cross_sample['Area'])
            comoving_width = min((Planck15.comoving_distance(sample['zeff']).value*Planck15.h)**2,
                                 (Planck15.comoving_distance(cross_sample['zeff']).value*Planck15.h)**2)
            area_comoving=area*(np.pi/180)**2*comoving_width
            
            vol2=area_comoving*(min(sample['depth'],cross_sample['depth'])/2) #not sure what this factor should be but 2 is close to 500 for lowz
            if(name is None): n_g2=sample['Ng']/vol2
            else: n_g2=sample['Ng_'+name]/vol
            g_shot_noise2=1./n_g2
            bzeff2 = 1.75/PS.DZ_int([cross_sample['zeff']])
            p_g2 = bzeff2**2 * cross_sample['pk']
            r,cov_gggg=HT.projected_covariance(k_pk=kh,pk1=p_g+g_shot_noise,pk2=p_g2+g_shot_noise2,j_nu=0,taper=True,**taper_kw)
        else:
            r,cov_gggg=HT.projected_covariance(k_pk=kh,pk1=p_g+g_shot_noise,pk2=p_g+g_shot_noise,j_nu=0,taper=True,**taper_kw)
        r_re,cov_gggg_re=HT.bin_cov(r=r,cov=cov_gggg,r_bins=r_bins)
        
        #\/ Is this right for cross? so long as we use overlap?
        cov_final_gg=cov_gggg_re*2./vol*Dchi2_gg #2 because there are 2 terms in the covariance, which are equal when galaxy sample is same.
        corr=HT.corr_matrix(cov=cov_final_gg)

        #compute SNR for the sample
        res = np.sqrt(wgg_re@np.linalg.inv(cov_final_gg)@wgg_re)
        if(name is None and cross is None): sample['snr'] = res
        elif(name is not None and cross is None): sample['snr_'+name] = res
        else: sample['snr_cross_'+cross]= res
            
            
    return None
run_snr()

In [151]:
#peep at results
sdict

{'lowz': {'zeff': 0.27,
  'Ng': 45671.0,
  'Area': 860.0,
  'depth': 1317.9232225273754,
  'pk': array([2.41539290e+03, 2.42018959e+03, 2.42499541e+03, ...,
         5.62995788e-01, 5.60238431e-01, 5.57494089e-01]),
  'sn': 2165.6772550291025,
  'snr': 20.08242170014838,
  'Ng_full_boss': 424846.511627907,
  'snr_full_boss': 61.25081196178207,
  'snr_cross_rm': 28.975828327917675},
 'cmass': {'zeff': 0.63,
  'Ng': 74186.0,
  'Area': 860.0,
  'depth': 981.8146241626717,
  'pk': array([1.67309651e+03, 1.67641989e+03, 1.67974959e+03, ...,
         3.31475451e-01, 3.29844697e-01, 3.28221687e-01]),
  'sn': 4452.071766905112,
  'snr': 26.040034520570057,
  'Ng_full_boss': 690102.3255813955,
  'snr_full_boss': 79.42136071597213,
  'snr_cross_rm': 22.68927950251981},
 'lrg': {'zeff': 0.8,
  'Ng': 24404.0,
  'Area': 700.0,
  'depth': 761.5328241925426,
  'pk': array([1.42224415e+03, 1.42506962e+03, 1.42790047e+03, ...,
         2.69852545e-01, 2.68525442e-01, 2.67204637e-01]),
  'sn': 15423.121

## 1. Total e/BOSS

In [142]:
boss_area = 8e3 #quoting Sukhdeep, use for all
#this is not the best way to do it but okay
for i_s,sname in enumerate(sdict): #have to run this in order or else complains about sdict too long
    sample = sdict[sname]
    area_comoving=boss_area*(np.pi/180)**2*(Planck15.comoving_distance(sample['zeff']).value*Planck15.h)**2
    vol=area_comoving*(depths[i_s]/2) #not sure what this factor should be but 2 is close to 500 for lowz 
    sample['Ng_full_boss'] =  1/(sample['sn']) * (vol)
    
#run everything again, just with the new area...
run_snr(area=boss_area,name='full_boss')
sdict

{'lowz': {'zeff': 0.27,
  'Ng': 45671.0,
  'Area': 860.0,
  'depth': 1317.9232225273754,
  'pk': array([2.41539290e+03, 2.42018959e+03, 2.42499541e+03, ...,
         5.62995788e-01, 5.60238431e-01, 5.57494089e-01]),
  'sn': 2165.6772550291025,
  'snr': 20.08242170014838,
  'Ng_full_boss': 424846.511627907,
  'snr_full_boss': 61.25081196178207},
 'cmass': {'zeff': 0.63,
  'Ng': 74186.0,
  'Area': 860.0,
  'depth': 981.8146241626717,
  'pk': array([1.67309651e+03, 1.67641989e+03, 1.67974959e+03, ...,
         3.31475451e-01, 3.29844697e-01, 3.28221687e-01]),
  'sn': 4452.071766905112,
  'snr': 26.040034520570057,
  'Ng_full_boss': 690102.3255813955,
  'snr_full_boss': 79.42136071597213},
 'lrg': {'zeff': 0.8,
  'Ng': 24404.0,
  'Area': 700.0,
  'depth': 761.5328241925426,
  'pk': array([1.42224415e+03, 1.42506962e+03, 1.42790047e+03, ...,
         2.69852545e-01, 2.68525442e-01, 2.67204637e-01]),
  'sn': 15423.1210686697,
  'snr': 13.862174874142339,
  'Ng_full_boss': 227013.95348837215,

## 2. Total redMaGiC

In [17]:
#0.0011247376138338568
#clustering redshift nbar within 10% of what is quoted in the redMaGic "fiducial sample" of https://arxiv.org/pdf/1507.05460.pdf

0.0011247376138338568

In [143]:
names.append('rm')
rm_area  = 5e3 #full DES footprint
rm_zrange = [0.14,0.94]
rm_zeff = np.mean(rm_zrange)
#area_comoving=rm_area*(np.pi/180)**2 *(Planck15.comoving_distance(rm_zeff).value*Planck15.h)**2
rm_depth = (Planck15.comoving_distance(rm_zrange[1]).value*Planck15.h #convert to Mpc/h
                   - Planck15.comoving_distance(rm_zrange[0]).value*Planck15.h) /2 #factor of 2 again...
#vol_comoving = rm_depth*area_comoving
rm_Ng =  3041935 #quoted in clustering redshifts paper 3.3
# n_g=rm_Ng/vol_comoving
# g_shot_noise=1./n_g
# p_g=(1.75/PS.DZ_int([rm_zeff]))**2 * pks[i_s]
rm_pk = np.squeeze(get_camb([rm_zeff])[2])

In [144]:
#add to dictionary
sdict['rm'] = {'zeff':rm_zeff, 'Area':rm_area, "Ng":rm_Ng,'depth':rm_depth,'pk':rm_pk}

In [145]:
run_snr()

In [146]:
sdict['rm']

{'zeff': 0.54,
 'Area': 5000.0,
 'Ng': 3041935,
 'depth': 895.8385886792394,
 'pk': array([1.82906120e+03, 1.83269415e+03, 1.83633401e+03, ...,
        3.73273533e-01, 3.71437609e-01, 3.69610395e-01]),
 'sn': 76.46227790573711,
 'snr': 49.73693932230535}

## 3. e/BOSS x redMaGiC

In [155]:
run_snr(cross='rm')

In [156]:
#note that rm cross rm is just itself
sdict

{'lowz': {'zeff': 0.27,
  'Ng': 45671.0,
  'Area': 860.0,
  'depth': 1317.9232225273754,
  'pk': array([2.41539290e+03, 2.42018959e+03, 2.42499541e+03, ...,
         5.62995788e-01, 5.60238431e-01, 5.57494089e-01]),
  'sn': 2165.6772550291025,
  'snr': 20.08242170014838,
  'Ng_full_boss': 424846.511627907,
  'snr_full_boss': 61.25081196178207,
  'snr_cross_rm': 23.519966590209652},
 'cmass': {'zeff': 0.63,
  'Ng': 74186.0,
  'Area': 860.0,
  'depth': 981.8146241626717,
  'pk': array([1.67309651e+03, 1.67641989e+03, 1.67974959e+03, ...,
         3.31475451e-01, 3.29844697e-01, 3.28221687e-01]),
  'sn': 4452.071766905112,
  'snr': 26.040034520570057,
  'Ng_full_boss': 690102.3255813955,
  'snr_full_boss': 79.42136071597213,
  'snr_cross_rm': 27.03735354649339},
 'lrg': {'zeff': 0.8,
  'Ng': 24404.0,
  'Area': 700.0,
  'depth': 761.5328241925426,
  'pk': array([1.42224415e+03, 1.42506962e+03, 1.42790047e+03, ...,
         2.69852545e-01, 2.68525442e-01, 2.67204637e-01]),
  'sn': 15423.121